In [104]:
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import re

In [11]:
opts= Options()
opts.add_experimental_option('excludeSwitches', ['enable-automation'])
opts.add_experimental_option('useAutomationExtension', False)
opts.add_argument('--start-maximized') 
opts.add_argument('user.data-dir=selenium') 
#opts.add_argument('drive/adblock.crx') 
opts.add_argument('--incognito')

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.espn.com/soccer/standings/_/league/ita.1')
driver.implicitly_wait(5)

#accept cookies:
driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()

standings = []

for i in range(2003, 2021):
    try:
        driver.get(f"https://www.espn.com/soccer/standings/_/league/ITA.1/season/{i}")
        tmp = driver.find_element_by_css_selector('#fittPageContainer > div:nth-child(4) > div > div > section > div > section > section > div.standings__table > div').get_attribute('outerHTML')
        standings.append(pd.read_html(tmp))
    except:
        standings.append('unavailable')



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/baccandres/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/xv/v29gqxhs4b5gth_lvdz9tgl80000gn/T/ipykernel_2727/1996998481.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
/var/folders/xv/v29gqxhs4b5gth_lvdz9tgl80000gn/T/ipykernel_2727/1996998481.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('#onetrust-accept-btn-handler').click()
/var/folders/xv/v29gqxhs4b5gth_lvdz9tgl80000gn/T/ipykernel_2727/1996998481.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tmp = driver.find_element_by_css_selector('#fittPageContainer > div:nth-child(4) > div > div > 

In [163]:
standings_dfs = []

for i in range(len(standings)):
    standings_dfs.append(pd.concat([standings[i][0], standings[0][1]], axis=1))

In [164]:
for i in range(len(standings_dfs)):
    for x in range(2003, 2021):
        standings_dfs[i]['Campaign'] = standings_dfs[i].columns[0][:4]

In [165]:
for i in range(len(standings_dfs)):
    standings_dfs[i].columns = ['Teams', 'GP', 'W', 'D', 'L', 'F','A', 'GD', 'P', 'Campaign']

In [166]:
df_standings = pd.concat(standings_dfs)

In [167]:
def acronym (x):
    return "".join(re.split('[^a-zA-Z]*', x))[:3]

In [168]:
df_standings['Teams'] = df_standings['Teams'].apply(acronym)

In [170]:
df_standings = df_standings[['Campaign','Teams', 'GP', 'W', 'D', 'L', 'F','A', 'GD', 'P']]
df_standings = df_standings.dropna().reset_index(drop=True)
df_standings.sample(3)

,Campaign,Teams,GP,W,D,L,F,A,GD,P
124,2009,BOL,34.0,7.0,9.0,18.0,26.0,54.0,-28.0,30.0
49,2005,CAG,34.0,8.0,10.0,16.0,41.0,54.0,-13.0,34.0
276,2018,TOR,34.0,13.0,11.0,10.0,44.0,40.0,4.0,50.0


In [171]:
df_standings.to_csv('Standings.csv')